# Data Analysis

# Notebook set-up

In [752]:
import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import io
import statistics as stat
import statsmodels.api as sm
from statsmodels.api import OLS

sns.set_style('whitegrid')
sns.set_context('notebook')

# Data loading

In [753]:
# Load the data into a dataframe 
#can_master = pd.read_csv('can-1996-2016.csv')
#cmte_master =pd.read_csv('cm-1996-2014.csv') 
#opex=pd.read_csv('opex-2004-2014.csv')
#oth=pd.read_csv('oth-1996-2014.csv')
#exp=pd.read_csv('expresults_rev.csv')
#election_spend=pd.read_csv('catall.csv')
#results = pd.read_csv('candr.csv')
#catsort= pd.read_csv('catsort_rev.csv')
cost_vote = pd.read_csv('combined_all.csv')

In [754]:
cost_vote.head()

,cand_id,cycle,c1,c2,c3,c4,c5,c6,c7,c8,...,incumbent,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,ge_winner_indicator,total_votes,totalspend,costpervote
0,H0AK00089,2010,550.00,0.00,0.00,0.00,0.0,785.00,0.00,0.0,...,0,DEM,38673.0,0.0,77606.0,0.0,0,116279.0,220001.06,1.892010
1,H0AK01038,2010,1569.23,88.93,0.00,0.00,0.0,0.00,0.00,0.0,...,0,REP,24709.0,0.0,0.0,0.0,0,24709.0,169777.74,6.871089
2,H0AL00016,2010,19831.34,909.80,81.19,465.00,0.0,3177.55,2192.99,0.0,...,0,DEM,6672.0,0.0,0.0,0.0,0,6672.0,26957.87,4.040448
3,H0AL01030,2010,7054.34,5233.32,668.95,27082.28,1175.0,1295.21,583.75,0.0,...,0,REP,18725.0,0.0,0.0,0.0,0,18725.0,44377.00,2.369933
4,H0AL01030,2012,3480.00,27.72,381.09,7257.71,0.0,0.00,56.58,0.0,...,0,R,3854.0,0.0,0.0,0.0,0,3854.0,11203.10,2.906876


In [796]:
cost_vote.dtypes

cand_id                      object
cycle                         int64
c1                          float64
c2                          float64
c3                          float64
c4                          float64
c5                          float64
c6                          float64
c7                          float64
c8                          float64
c9                          float64
c10                         float64
c11                         float64
c12                         float64
c13                         float64
state_abbreviation           object
district                     object
incumbent                    object
party                        object
primary_votes               float64
runoff_votes                float64
general_votes               float64
ge_runoff_election_votes    float64
winner                       object
total_votes                 float64
totalspend                  float64
costvote                    float64
dtype: object

In [755]:
cost_vote['total_votes']=cost_vote['total_votes'].fillna(0)

In [756]:
cost_vote['totalspend']=cost_vote['totalspend'].fillna(0)

In [757]:
cost_vote['incumbent']=cost_vote['incumbent'].fillna(0)

In [758]:
cost_vote['party']=cost_vote['party'].fillna('None')

In [759]:
cost_vote = cost_vote.drop(['costpervote'], axis=1)

In [760]:
cost_vote['totalspend']=pd.to_numeric(cost_vote['totalspend'],errors='coerce')

In [761]:
print(cost_vote[pd.to_numeric(cost_vote['totalspend'], errors = 'coerce').isnull()])

Empty DataFrame
Columns: [cand_id, cycle, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, state_abbreviation, district, incumbent, party, primary_votes, runoff_votes, general_votes, ge_runoff_election_votes, ge_winner_indicator, total_votes, totalspend]
Index: []

[0 rows x 26 columns]


In [762]:
cost_vote['total_votes']=pd.to_numeric(cost_vote['total_votes'],errors='coerce')

In [763]:
print(cost_vote[pd.to_numeric(cost_vote['total_votes'], errors = 'coerce').isnull()])

Empty DataFrame
Columns: [cand_id, cycle, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, state_abbreviation, district, incumbent, party, primary_votes, runoff_votes, general_votes, ge_runoff_election_votes, ge_winner_indicator, total_votes, totalspend]
Index: []

[0 rows x 26 columns]


In [799]:
cost_vote['cycle']=pd.to_numeric(cost_vote['cycle'],errors='coerce')

In [800]:
print(cost_vote[pd.to_numeric(cost_vote['cycle'], errors = 'coerce').isnull()])

Empty DataFrame
Columns: [cand_id, cycle, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, state_abbreviation, district, incumbent, party, primary_votes, runoff_votes, general_votes, ge_runoff_election_votes, winner, total_votes, totalspend, costvote]
Index: []

[0 rows x 27 columns]


In [801]:
cost_vote = cost_vote[cost_vote.total_votes != 0]

In [802]:
cost_vote.loc[cost_vote['totalspend'] == 0]

,cand_id,cycle,c1,c2,c3,c4,c5,c6,c7,c8,...,incumbent,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,winner,total_votes,totalspend,costvote
9107,H4TN09055,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,D,122.0,0.0,0.0,0.0,0,122.0,0.0,0.0
9853,H6VA10191,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,IND,0.0,0.0,1851.0,0.0,0,1851.0,0.0,0.0
10982,S4MD00160,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,R,4110.0,0.0,0.0,0.0,0,4110.0,0.0,0.0
11505,H6LA02207,2008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,L,0.0,0.0,549.0,0.0,0,549.0,0.0,0.0
11506,H8NJ02133,2008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,RTB,0.0,0.0,1312.0,0.0,0,1312.0,0.0,0.0
11507,H0NJ02098,2008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,SUS,0.0,0.0,648.0,0.0,0,648.0,0.0,0.0
11508,H6RI01104,2008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,I,0.0,0.0,15108.0,0.0,0,15108.0,0.0,0.0
11509,H4AL02063,2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,D,0.0,0.0,70562.0,0.0,0,70562.0,0.0,0.0
11510,H2AL05078,2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,D,4393.0,0.0,0.0,0.0,0,4393.0,0.0,0.0
11511,H4AL05066,2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,R,6742.0,2371.0,74145.0,0.0,0,83258.0,0.0,0.0


In [803]:
cost_vote = cost_vote.rename(columns = {'ge_winner_indicator':'winner'})

In [804]:
cost_vote['costvote'] = cost_vote['totalspend']/cost_vote['total_votes']

In [768]:
cost_vote.mean()

cycle                         2009.142466
c1                           82813.309249
c2                            2937.686044
c3                           24862.413532
c4                           77011.650229
c5                            4297.670139
c6                            7494.260491
c7                            2807.764581
c8                             115.423473
c9                              45.622999
c10                             83.920067
c11                            748.612873
c12                            287.573776
c13                         402798.463396
primary_votes                20683.139878
runoff_votes                   441.977017
general_votes                76868.068037
ge_runoff_election_votes       232.190030
total_votes                  98225.374962
totalspend                  606188.947376
costvote                      2662.593130
dtype: float64

In [806]:
cost_vote['cycle'] = cost_vote['cycle'].astype(int).astype('str')

In [808]:
cost_vote.describe()

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,total_votes,totalspend,costvote
count,1.314000e+04,13140.000000,1.314000e+04,1.314000e+04,13140.000000,1.314000e+04,1.314000e+04,13140.000000,13140.000000,13140.000000,13140.000000,13140.000000,1.314000e+04,1.314000e+04,13140.000000,1.314000e+04,1.314000e+04,1.314000e+04,1.314000e+04,1.314000e+04
mean,8.281331e+04,2937.686044,2.486241e+04,7.701165e+04,4297.670139,7.494260e+03,2.807765e+03,115.423473,45.622999,83.920067,748.612873,287.573776,4.027985e+05,2.068314e+04,441.977017,7.686807e+04,2.321900e+02,9.822537e+04,6.061889e+05,2.662593e+03
std,4.662288e+05,13721.331612,1.365725e+05,5.160868e+05,19948.936705,4.880747e+04,2.082310e+04,5068.014913,875.780998,2216.931541,8543.058359,3322.745012,1.545528e+06,6.919007e+04,8472.866811,2.632518e+05,1.354564e+04,3.218754e+05,1.821591e+06,1.123215e+05
min,-7.637000e+02,-390.000000,-1.900000e-01,-7.500000e+02,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,-5000.000000,0.000000,0.000000,-4.920380e+04,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,4.663000e+03,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,2.125160e+03,5.676000e+03,0.000000,6.505000e+03,0.000000e+00,1.617900e+04,3.202793e+04,1.441572e+00
75%,1.417877e+04,68.235000,2.014810e+03,4.148600e+03,0.000000,3.873775e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,1.517188e+05,2.091725e+04,0.000000,1.013965e+05,0.000000e+00,1.238395e+05,5.978935e+05,7.717508e+00
max,2.545184e+07,527439.630000,8.283133e+06,2.546524e+07,777330.090000,2.290329e+06,1.000230e+06,500000.000000,43129.070000,164844.000000,407000.000000,117400.000000,3.943196e+07,2.566298e+06,631812.000000,7.864624e+06,1.228033e+06,1.025745e+07,4.955574e+07,7.044762e+06


In [809]:
cost_vote['party'] = cost_vote['party'].astype('str')

In [810]:
cost_vote = cost_vote.replace("DEM","D")

In [811]:
cost_vote = cost_vote.replace("REP","R")

In [812]:
cost_vote = cost_vote.replace("LBT","L")

In [813]:
cost_vote = cost_vote.replace("LIB","L")

In [814]:
major = !=["R","D","L"]

In [815]:
cost_vote = cost_vote.replace(major,"O")

In [816]:
cost_vote.head()

,cand_id,cycle,c1,c2,c3,c4,c5,c6,c7,c8,...,incumbent,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,winner,total_votes,totalspend,costvote
0,H0AK00089,2010,550.00,0.00,0.00,0.00,0.0,785.00,0.00,0.0,...,0,D,38673.0,0.0,77606.0,0.0,0,116279.0,220001.06,1.892010
1,H0AK01038,2010,1569.23,88.93,0.00,0.00,0.0,0.00,0.00,0.0,...,0,R,24709.0,0.0,0.0,0.0,0,24709.0,169777.74,6.871089
2,H0AL00016,2010,19831.34,909.80,81.19,465.00,0.0,3177.55,2192.99,0.0,...,0,D,6672.0,0.0,0.0,0.0,0,6672.0,26957.87,4.040448
3,H0AL01030,2010,7054.34,5233.32,668.95,27082.28,1175.0,1295.21,583.75,0.0,...,0,R,18725.0,0.0,0.0,0.0,0,18725.0,44377.00,2.369933
4,H0AL01030,2012,3480.00,27.72,381.09,7257.71,0.0,0.00,56.58,0.0,...,0,R,3854.0,0.0,0.0,0.0,0,3854.0,11203.10,2.906876


In [817]:
cost_vote['party'].count()

13140

In [818]:
cost_vote['winner'].value_counts()

0             10355
1              2781
W (Runoff)        2
Dd                1
WW                1
Name: winner, dtype: int64

## Scores for Democrats (blue font unintentional) 

In [823]:
Gen = cost_vote[cost_vote.general_votes != 0]

In [824]:
Dem = Gen[Gen['party'].isin(['D'])]

In [825]:
Dem.head(100)

,cand_id,cycle,c1,c2,c3,c4,c5,c6,c7,c8,...,incumbent,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,winner,total_votes,totalspend,costvote
0,H0AK00089,2010,550.00,0.00,0.00,0.00,0.00,785.00,0.00,0.0,...,0,D,38673.0,0.0,77606.0,0.0,0,116279.0,220001.06,1.892010
9,H0AL03184,2010,14.95,1441.91,0.00,5000.00,0.00,2293.50,0.00,0.0,...,0,D,0.0,0.0,80204.0,0.0,0,80204.0,8750.36,0.109101
19,H0AL07086,2010,717735.35,38988.03,52236.78,690997.05,55500.00,50828.50,31230.29,0.0,...,0,D,31531.0,32366.0,136696.0,0.0,1,200593.0,1673438.30,8.342456
20,H0AL07086,2012,329788.25,48925.15,65657.68,23994.00,0.00,40971.17,140490.99,0.0,...,1,D,0.0,0.0,232520.0,0.0,1,232520.0,661131.32,2.843331
21,H0AL07086,2014,425989.76,57266.94,116612.17,303683.52,30000.00,58571.83,166933.55,0.0,...,1,D,74953.0,0.0,133687.0,0.0,1,208640.0,1165162.70,5.584560
27,H0AR01109,2010,70562.30,5872.99,17720.93,303707.61,26000.00,6671.16,829.76,0.0,...,0,D,25854.0,39402.0,78267.0,0.0,0,143523.0,1562558.79,10.887166
47,H0AR03030,2010,31154.10,784.00,8468.11,4742.09,395.79,3535.65,217.35,0.0,...,0,D,0.0,0.0,56542.0,0.0,0,56542.0,74834.03,1.323512
60,H0AS00018,2004,1192.21,0.00,2021.58,8346.75,0.00,310.00,0.00,0.0,...,1,D,0.0,0.0,6656.0,0.0,1,6656.0,104193.08,15.654008
104,H0CA02138,2010,29435.14,2654.79,2552.37,46165.70,16750.00,8365.14,6036.95,0.0,...,0,D,42334.0,0.0,98092.0,0.0,0,140426.0,131000.68,0.932881
105,H0CA02138,2012,64561.45,7306.04,288.00,26013.00,2960.00,144.22,80.00,0.0,...,0,D,43409.0,0.0,125386.0,0.0,0,168795.0,177103.32,1.049221


In [826]:
DemW = Dem[Dem['winner'].isin(['1'])] #table of Democrats who won
DemW.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,1267.0,1.199751e+05,3.508850e+05,0.00,0.000000,0.000000,1.168401e+05,7.071784e+06
c2,1267.0,6.036074e+03,1.999258e+04,0.00,0.000000,0.000000,2.665975e+03,2.893896e+05
c3,1267.0,4.683261e+04,1.871672e+05,0.00,0.000000,0.000000,4.140319e+04,5.594799e+06
c4,1267.0,1.444014e+05,9.467859e+05,-750.00,0.000000,0.000000,5.594320e+03,2.546524e+07
c5,1267.0,9.649582e+03,4.169237e+04,0.00,0.000000,0.000000,0.000000e+00,7.773301e+05
c6,1267.0,1.538446e+04,8.322974e+04,0.00,0.000000,0.000000,1.006340e+03,2.290329e+06
c7,1267.0,6.245743e+03,3.424702e+04,0.00,0.000000,0.000000,9.900000e+02,1.000230e+06
c8,1267.0,2.807535e+02,5.966891e+03,0.00,0.000000,0.000000,0.000000e+00,1.800000e+05
c9,1267.0,1.164275e+02,1.610966e+03,0.00,0.000000,0.000000,0.000000e+00,4.312907e+04
c10,1267.0,7.386825e+01,5.182740e+02,-400.00,0.000000,0.000000,0.000000e+00,9.300000e+03


In [827]:
DemWInc = DemW[DemW['incumbent'].isin(['1'])] #table of Democrat Incumbent who won
DemWInc.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,1082.0,1.027144e+05,2.674979e+05,0.00,0.000000,0.000000,1.029508e+05,4.238332e+06
c2,1082.0,5.974485e+03,2.055652e+04,0.00,0.000000,0.000000,2.377308e+03,2.893896e+05
c3,1082.0,4.301155e+04,1.026644e+05,0.00,0.000000,0.000000,4.165430e+04,1.404384e+06
c4,1082.0,9.161149e+04,5.470340e+05,-750.00,0.000000,0.000000,3.792497e+03,9.983008e+06
c5,1082.0,6.393242e+03,2.851137e+04,0.00,0.000000,0.000000,0.000000e+00,5.923005e+05
c6,1082.0,8.534720e+03,3.586878e+04,0.00,0.000000,0.000000,4.486450e+02,5.160952e+05
c7,1082.0,6.306496e+03,3.639518e+04,0.00,0.000000,0.000000,3.500000e+02,1.000230e+06
c8,1082.0,2.352132e+02,5.683070e+03,0.00,0.000000,0.000000,0.000000e+00,1.800000e+05
c9,1082.0,1.101515e+02,1.599351e+03,0.00,0.000000,0.000000,0.000000e+00,4.312907e+04
c10,1082.0,5.944092e+01,4.648726e+02,-400.00,0.000000,0.000000,0.000000e+00,9.300000e+03


In [830]:
DemWC = DemW[DemW['incumbent'].isin(['0'])] #table of Democrat challengers who won
DemWC.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,185.0,2.209269e+05,6.440000e+05,0.0,0.000000e+00,1.670529e+04,2.535038e+05,7.071784e+06
c2,185.0,6.396287e+03,1.635166e+04,0.0,0.000000e+00,0.000000e+00,6.115970e+03,1.713475e+05
c3,185.0,6.918062e+04,4.225187e+05,0.0,0.000000e+00,2.735900e+03,3.996882e+04,5.594799e+06
c4,185.0,4.531512e+05,2.073003e+06,0.0,0.000000e+00,0.000000e+00,2.035845e+05,2.546524e+07
c5,185.0,2.869477e+04,8.220168e+04,0.0,0.000000e+00,0.000000e+00,3.581110e+04,7.773301e+05
c6,185.0,5.544621e+04,1.954838e+05,0.0,0.000000e+00,0.000000e+00,3.039067e+04,2.290329e+06
c7,185.0,5.890416e+03,1.695737e+04,0.0,0.000000e+00,0.000000e+00,4.669500e+03,1.877644e+05
c8,185.0,5.471027e+02,7.425599e+03,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.010000e+05
c9,185.0,1.531336e+02,1.681307e+03,0.0,0.000000e+00,0.000000e+00,0.000000e+00,2.218561e+04
c10,185.0,1.582486e+02,7.550699e+02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,8.313270e+03


In [831]:
DemL = Dem[Dem['winner'].isin(['0'])] #table of Democrats who won
DemL.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,1327.0,74627.974295,2.653480e+05,0.0,0.000000,885.480000,37941.670000,4.360481e+06
c2,1327.0,3059.714936,1.270278e+04,-390.0,0.000000,0.000000,1611.315000,2.932756e+05
c3,1327.0,14895.315418,5.258994e+04,0.0,0.000000,0.000000,6039.150000,7.080864e+05
c4,1327.0,95057.723595,3.959304e+05,-324.0,0.000000,0.000000,17048.250000,8.772151e+06
c5,1327.0,6155.928184,1.985510e+04,0.0,0.000000,0.000000,0.000000,2.048548e+05
c6,1327.0,11311.573738,5.881164e+04,0.0,0.000000,0.000000,5520.820000,1.534658e+06
c7,1327.0,1958.495961,9.088393e+03,0.0,0.000000,0.000000,673.625000,1.525661e+05
c8,1327.0,20.481959,3.992992e+02,0.0,0.000000,0.000000,0.000000,1.137500e+04
c9,1327.0,27.953851,2.709405e+02,0.0,0.000000,0.000000,0.000000,5.626030e+03
c10,1327.0,99.073534,1.468812e+03,0.0,0.000000,0.000000,0.000000,4.983207e+04


In [832]:
DemLInc = DemL[DemL['incumbent'].isin(['1'])] #table of Democrat Incumbent who lose
DemLInc.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,134.0,2.007616e+05,5.297954e+05,0.0,0.000000,0.000000e+00,1.622017e+05,4.104223e+06
c2,134.0,7.777744e+03,2.476687e+04,-390.0,0.000000,0.000000e+00,7.730797e+03,2.564344e+05
c3,134.0,5.197178e+04,1.122384e+05,0.0,0.000000,0.000000e+00,5.594996e+04,6.066072e+05
c4,134.0,2.737383e+05,9.412709e+05,0.0,0.000000,0.000000e+00,5.718943e+03,8.772151e+06
c5,134.0,1.493568e+04,3.702865e+04,0.0,0.000000,0.000000e+00,0.000000e+00,2.048548e+05
c6,134.0,1.590541e+04,5.779092e+04,0.0,0.000000,0.000000e+00,3.140160e+03,5.585654e+05
c7,134.0,5.183430e+03,1.859039e+04,0.0,0.000000,0.000000e+00,1.005745e+03,1.493258e+05
c8,134.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
c9,134.0,2.985022e+01,2.413234e+02,0.0,0.000000,0.000000e+00,0.000000e+00,2.000000e+03
c10,134.0,5.183029e+02,4.356871e+03,0.0,0.000000,0.000000e+00,0.000000e+00,4.983207e+04


In [833]:
DemLC = DemL[DemL['incumbent'].isin(['0'])] #table of Democrat Incumbent who lose
DemLC.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,1193.0,60460.410017,2.121733e+05,0.0,0.000000,1172.170000,35016.330000,4.360481e+06
c2,1193.0,2529.777091,1.040555e+04,0.0,0.000000,0.000000,1498.850000,2.932756e+05
c3,1193.0,10730.817477,3.871861e+04,0.0,0.000000,0.000000,4796.660000,7.080864e+05
c4,1193.0,74987.981098,2.674599e+05,-324.0,0.000000,0.000000,18038.170000,4.025618e+06
c5,1193.0,5169.770319,1.661077e+04,0.0,0.000000,0.000000,0.000000,1.923870e+05
c6,1193.0,10795.585809,5.892666e+04,0.0,0.000000,0.000000,5691.910000,1.534658e+06
c7,1193.0,1596.265272,7.212662e+03,0.0,0.000000,0.000000,641.910000,1.525661e+05
c8,1193.0,22.782531,4.210831e+02,0.0,0.000000,0.000000,0.000000,1.137500e+04
c9,1193.0,27.740847,2.741582e+02,0.0,0.000000,0.000000,0.000000,5.626030e+03
c10,1193.0,51.984904,5.098685e+02,0.0,0.000000,0.000000,0.000000,1.140000e+04


In [834]:
Rep = Gen[Gen['party'].isin(['R'])]

In [835]:
Rep.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,2579.0,2.072966e+05,8.460947e+05,-0.29,0.000000,5590.320000,1.601081e+05,2.545184e+07
c2,2579.0,6.304956e+03,1.994414e+04,0.00,0.000000,0.000000,2.978155e+03,5.274396e+05
c3,2579.0,6.604212e+04,2.410546e+05,0.00,0.000000,298.250000,4.423546e+04,8.283133e+06
c4,2579.0,1.396537e+05,5.996801e+05,0.00,0.000000,0.000000,4.327663e+04,1.811699e+07
c5,2579.0,7.637804e+03,2.223311e+04,0.00,0.000000,0.000000,2.225000e+02,3.169784e+05
c6,2579.0,1.084493e+04,5.548636e+04,0.00,0.000000,0.000000,2.960750e+03,1.872233e+06
c7,2579.0,5.973010e+03,2.679677e+04,0.00,0.000000,0.000000,2.000000e+03,8.582995e+05
c8,2579.0,4.126891e+02,1.062226e+04,0.00,0.000000,0.000000,0.000000e+00,5.000000e+05
c9,2579.0,5.368474e+01,1.020658e+03,0.00,0.000000,0.000000,0.000000e+00,3.620453e+04
c10,2579.0,1.314378e+02,3.583009e+03,-5000.00,0.000000,0.000000,0.000000e+00,1.648440e+05


In [836]:
RepW = Rep[Rep['winner'].isin(['1'])]
RepW.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,1326.0,2.577024e+05,5.937132e+05,0.00,0.000000,23237.710000,3.063876e+05,8.853032e+06
c2,1326.0,8.815329e+03,2.433561e+04,0.00,0.000000,0.000000,7.877342e+03,5.274396e+05
c3,1326.0,8.879124e+04,2.018838e+05,0.00,0.000000,5158.070000,1.081441e+05,3.066642e+06
c4,1326.0,1.500616e+05,4.929795e+05,0.00,0.000000,0.000000,8.852273e+04,1.223820e+07
c5,1326.0,9.903480e+03,2.497333e+04,0.00,0.000000,0.000000,6.900000e+03,3.169784e+05
c6,1326.0,9.848562e+03,3.401812e+04,0.00,0.000000,0.000000,2.182648e+03,4.990009e+05
c7,1326.0,7.526809e+03,2.159333e+04,0.00,0.000000,0.000000,3.860815e+03,3.121001e+05
c8,1326.0,6.898546e+02,1.443028e+04,0.00,0.000000,0.000000,0.000000e+00,5.000000e+05
c9,1326.0,8.358416e+01,1.376192e+03,0.00,0.000000,0.000000,0.000000e+00,3.620453e+04
c10,1326.0,2.375463e+02,4.987876e+03,0.00,0.000000,0.000000,0.000000e+00,1.648440e+05


In [837]:
RepWInc = RepW[RepW['incumbent'].isin(['1'])]
RepWInc.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,1093.0,2.398409e+05,5.518194e+05,0.00,0.000000,13000.870000,2.809910e+05,8.853032e+06
c2,1093.0,8.214321e+03,2.020866e+04,0.00,0.000000,0.000000,7.097140e+03,1.804328e+05
c3,1093.0,8.835692e+04,1.975340e+05,0.00,0.000000,4055.500000,1.110198e+05,3.066642e+06
c4,1093.0,1.033081e+05,2.837885e+05,0.00,0.000000,0.000000,5.020331e+04,2.836189e+06
c5,1093.0,8.611447e+03,2.254651e+04,0.00,0.000000,0.000000,1.821990e+03,3.169784e+05
c6,1093.0,8.593977e+03,3.286661e+04,0.00,0.000000,0.000000,1.551450e+03,4.990009e+05
c7,1093.0,7.530983e+03,2.219011e+04,0.00,0.000000,0.000000,3.677850e+03,3.121001e+05
c8,1093.0,8.038643e+02,1.587516e+04,0.00,0.000000,0.000000,0.000000e+00,5.000000e+05
c9,1093.0,8.265757e+01,1.461936e+03,0.00,0.000000,0.000000,0.000000e+00,3.620453e+04
c10,1093.0,2.186181e+02,5.076361e+03,0.00,0.000000,0.000000,0.000000e+00,1.648440e+05


In [838]:
RepWC = Rep[Rep['incumbent'].isin(['0'])]
RepWC.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,1388.0,175310.494265,1.034101e+06,-0.29,0.000000,3331.000000,7.522167e+04,2.545184e+07
c2,1388.0,4298.117774,1.800384e+04,0.00,0.000000,0.000000,1.499310e+03,5.274396e+05
c3,1388.0,38860.785101,1.580284e+05,0.00,0.000000,0.000000,1.305766e+04,3.518841e+06
c4,1388.0,148195.179769,7.242292e+05,0.00,0.000000,0.000000,3.697907e+04,1.811699e+07
c5,1388.0,6135.219733,1.950180e+04,0.00,0.000000,0.000000,0.000000e+00,2.734362e+05
c6,1388.0,12063.845187,6.716189e+04,0.00,0.000000,0.000000,4.421440e+03,1.872233e+06
c7,1388.0,4282.806744,2.707683e+04,0.00,0.000000,0.000000,1.287438e+03,8.582995e+05
c8,1388.0,31.969726,7.030291e+02,0.00,0.000000,0.000000,0.000000e+00,2.279056e+04
c9,1388.0,34.660101,5.023567e+02,0.00,0.000000,0.000000,0.000000e+00,1.168465e+04
c10,1388.0,67.340367,1.883028e+03,-5000.00,0.000000,0.000000,0.000000e+00,6.953224e+04


In [839]:
RepL = Rep[Rep['winner'].isin(['0'])]
RepL.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,1250.0,153246.067240,1.047505e+06,-0.29,0.000000,2163.935000,49467.530000,2.545184e+07
c2,1250.0,3612.894088,1.332654e+04,0.00,0.000000,0.000000,1050.845000,2.365198e+05
c3,1250.0,41690.754696,2.746369e+05,0.00,0.000000,0.000000,9302.522500,8.283133e+06
c4,1250.0,126270.019472,6.914690e+05,0.00,0.000000,0.000000,28787.982500,1.811699e+07
c5,1250.0,5083.717824,1.784564e+04,0.00,0.000000,0.000000,0.000000,2.207500e+05
c6,1250.0,11862.138128,7.156846e+04,0.00,0.000000,0.000000,3948.235000,1.872233e+06
c7,1250.0,4297.205752,3.132184e+04,0.00,0.000000,0.000000,859.750000,8.582995e+05
c8,1250.0,119.662464,3.438098e+03,0.00,0.000000,0.000000,0.000000,1.202947e+05
c9,1250.0,22.096272,3.729753e+02,0.00,0.000000,0.000000,0.000000,1.150000e+04
c10,1250.0,19.193424,2.843237e+02,-5000.00,0.000000,0.000000,0.000000,5.000000e+03


In [841]:
RepLInc = RepL[RepL['incumbent'].isin(['1'])]
RepLInc.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,96.0,2.783346e+05,5.101413e+05,0.0,0.000000e+00,2.352942e+04,3.298094e+05,2.302124e+06
c2,96.0,1.293684e+04,3.465435e+04,0.0,0.000000e+00,0.000000e+00,1.155245e+04,2.365198e+05
c3,96.0,1.997231e+05,8.528603e+05,0.0,0.000000e+00,8.647240e+03,1.974637e+05,8.283133e+06
c4,96.0,3.690383e+05,9.617031e+05,0.0,0.000000e+00,0.000000e+00,1.353513e+05,7.191944e+06
c5,96.0,1.438836e+04,3.456190e+04,0.0,0.000000e+00,0.000000e+00,0.000000e+00,2.207500e+05
c6,96.0,1.820664e+04,7.143082e+04,0.0,0.000000e+00,0.000000e+00,8.115100e+02,5.913480e+05
c7,96.0,1.170751e+04,5.451083e+04,0.0,0.000000e+00,0.000000e+00,3.531590e+03,5.009458e+05
c8,96.0,1.472162e+03,1.236015e+04,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.202947e+05
c9,96.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
c10,96.0,6.833542e+01,4.981880e+02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,4.400000e+03


In [842]:
RepLC = RepL[RepL['incumbent'].isin(['0'])]
RepLC.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,1153.0,141877.161240,1.079682e+06,-0.29,0.000000,1689.540000,41575.360000,2.545184e+07
c2,1153.0,2813.710364,9.221109e+03,0.00,0.000000,0.000000,866.960000,1.177486e+05
c3,1153.0,28417.237580,1.397162e+05,0.00,0.000000,0.000000,7861.050000,3.518841e+06
c4,1153.0,103328.784458,6.542873e+05,0.00,0.000000,0.000000,26138.810000,1.811699e+07
c5,1153.0,4145.517988,1.441653e+04,0.00,0.000000,0.000000,0.000000,1.651940e+05
c6,1153.0,11332.446886,7.161650e+04,0.00,0.000000,0.000000,4027.840000,1.872233e+06
c7,1153.0,3629.130772,2.847086e+04,0.00,0.000000,0.000000,781.120000,8.582995e+05
c8,1153.0,7.155690,2.284881e+02,0.00,0.000000,0.000000,0.000000,7.753680e+03
c9,1153.0,23.955195,3.883031e+02,0.00,0.000000,0.000000,0.000000,1.150000e+04
c10,1153.0,15.118456,2.587691e+02,-5000.00,0.000000,0.000000,0.000000,5.000000e+03


In [843]:
Lib = cost_vote[cost_vote['party'].isin(['L'])]

In [844]:
Lib.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,889.0,879.089359,14044.488988,0.0,0.0,0.0,0.0,2.958971e+05
c2,889.0,92.602092,1505.483201,0.0,0.0,0.0,0.0,3.177802e+04
c3,889.0,775.674196,20495.719610,0.0,0.0,0.0,0.0,6.101976e+05
c4,889.0,1694.625411,36426.028055,0.0,0.0,0.0,0.0,1.077713e+06
c5,889.0,48.653543,1134.645189,0.0,0.0,0.0,0.0,3.310000e+04
c6,889.0,117.286299,1166.336665,0.0,0.0,0.0,0.0,3.050309e+04
c7,889.0,46.575917,564.495003,0.0,0.0,0.0,0.0,1.083557e+04
c8,889.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000e+00
c9,889.0,0.433780,12.933615,0.0,0.0,0.0,0.0,3.856300e+02
c10,889.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000e+00


In [847]:
LibW = Lib[Lib['winner'].isin(['1'])]
LibW.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c6,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c7,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c9,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c10,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [848]:
LibL = Lib[Lib['winner'].isin(['0'])]
LibL.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,889.0,879.089359,14044.488988,0.0,0.0,0.0,0.0,2.958971e+05
c2,889.0,92.602092,1505.483201,0.0,0.0,0.0,0.0,3.177802e+04
c3,889.0,775.674196,20495.719610,0.0,0.0,0.0,0.0,6.101976e+05
c4,889.0,1694.625411,36426.028055,0.0,0.0,0.0,0.0,1.077713e+06
c5,889.0,48.653543,1134.645189,0.0,0.0,0.0,0.0,3.310000e+04
c6,889.0,117.286299,1166.336665,0.0,0.0,0.0,0.0,3.050309e+04
c7,889.0,46.575917,564.495003,0.0,0.0,0.0,0.0,1.083557e+04
c8,889.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000e+00
c9,889.0,0.433780,12.933615,0.0,0.0,0.0,0.0,3.856300e+02
c10,889.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000e+00


In [850]:
LibLInc = LibL[LibL['incumbent'].isin(['1'])]
LibLInc.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c6,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c7,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c9,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c10,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [851]:
LibLC = LibL[LibL['incumbent'].isin(['0'])]
LibLC.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
c1,889.0,879.089359,14044.488988,0.0,0.0,0.0,0.0,2.958971e+05
c2,889.0,92.602092,1505.483201,0.0,0.0,0.0,0.0,3.177802e+04
c3,889.0,775.674196,20495.719610,0.0,0.0,0.0,0.0,6.101976e+05
c4,889.0,1694.625411,36426.028055,0.0,0.0,0.0,0.0,1.077713e+06
c5,889.0,48.653543,1134.645189,0.0,0.0,0.0,0.0,3.310000e+04
c6,889.0,117.286299,1166.336665,0.0,0.0,0.0,0.0,3.050309e+04
c7,889.0,46.575917,564.495003,0.0,0.0,0.0,0.0,1.083557e+04
c8,889.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000e+00
c9,889.0,0.433780,12.933615,0.0,0.0,0.0,0.0,3.856300e+02
c10,889.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000e+00


In [181]:
catsort['cycle'] = catsort['cycle'].astype(int).astype('str')

In [182]:
catsort.head()

,cand_id,cycle,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,total
0,H6FL19038,2006,281886.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H2NY17071,2004,4500.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H8TX27023,2008,12900.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H8OH18088,2012,30.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H0SC01071,2008,83183.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [183]:
exp['cycle']=pd.to_numeric(exp['cycle'],errors='coerce')

In [184]:
print(exp[pd.to_numeric(exp['cycle'], errors = 'coerce').isnull()])

       state_abbreviation district incumbent party  primary_votes  \
187                    CA        4       (I)   REP        63731.0   
188                    CA        4       (I)     R       105015.0   
264                    CA       45       NaN     R         8422.0   
282                    CA       48       NaN     D        39385.0   
283                    CA       48       NaN     D        39385.0   
284                    CA       49       (I)     R        56558.0   
285                    CA       49       (I)     R        71329.0   
286                    CA       49       (I)   REP        60447.0   
287                    CA       49       (I)     R        41369.0   
288                    CA       49       (I)   REP        47693.0   
289                    CA       49       (I)     R        66710.0   
1063                   ID        1       (I)     R        48986.0   
1064                   ID        1       (I)     R        48986.0   
1081                   ID        1

In [185]:
exp = exp[pd.notnull(exp['cycle'])]

In [186]:
exp['cycle'] = exp['cycle'].astype(int).astype('str')

In [187]:
exp.head()

,state_abbreviation,district,incumbent,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,ge_winner_indicator,total_votes,cand_id,cmte_id,exp_sum,category,cycle
0,AL,7,(I),D,0.0,NaN,228518.0,NaN,W,228518,H0AL07060,C00347872,1363.96,NaN,2012
1,AL,7,(I),DEM,93586.0,NaN,133870.0,NaN,W,227456,H0AL07060,C00347872,1363.96,NaN,2012
2,AL,7,(I),D,58193.0,NaN,183408.0,NaN,W,241601,H0AL07060,C00347872,1363.96,NaN,2012
3,AL,7,(I),D,0.0,NaN,228518.0,NaN,W,228518,H0AL07060,C00347872,1204248.81,NaN,2010
4,AL,7,(I),DEM,93586.0,NaN,133870.0,NaN,W,227456,H0AL07060,C00347872,1204248.81,NaN,2010


In [188]:
catsort.head()

,cand_id,cycle,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,total
0,H6FL19038,2006,281886.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H2NY17071,2004,4500.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H8TX27023,2008,12900.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H8OH18088,2012,30.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H0SC01071,2008,83183.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
#catsort['cycle']=catsort['cycle'].astype('str')

In [190]:
print(catsort.dtypes)

cand_id     object
cycle       object
c1         float64
c2         float64
c3         float64
c4         float64
c5         float64
c6         float64
c7         float64
c8         float64
c9         float64
c10        float64
c11        float64
c12        float64
c13        float64
total      float64
dtype: object


In [191]:
cs1 = catsort

In [192]:
cs1 = cs1.drop(['c2','c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis=1)

In [193]:
cs1.head()

,cand_id,cycle,c1
0,H6FL19038,2006,281886.64
1,H2NY17071,2004,4500.00
2,H8TX27023,2008,12900.00
3,H8OH18088,2012,30.00
4,H0SC01071,2008,83183.76


In [194]:
cs1.set_index(['cand_id','cycle'], inplace=True)

In [195]:
print(cs1)

                         c1
cand_id   cycle            
H6FL19038 2006    281886.64
H2NY17071 2004      4500.00
H8TX27023 2008     12900.00
H8OH18088 2012        30.00
H0SC01071 2008     83183.76
S0CA00199 2010     65563.32
S0MI00056 2014     49863.62
H2GA12113 2012    162924.41
H8NJ03156 2010     48652.69
H6AZ08020 2006     21606.11
H4WV03070 2014      2826.00
H4WI04191 2006       966.06
H8VA10072 2008      4602.00
H2MA01202 2012     13388.58
H6WV03083 2006      1652.00
H8WY00148 2012     68895.65
H8WI01024 2014    356582.98
H6TX09033 2004   1143785.48
H0IN07157 2012     28824.84
H2MI11042 2014      1938.75
H0FL12150 2010     11395.96
H8AL05109 2012     10019.08
H8VA11039 2010      6902.86
H2MN08103 2012     22125.35
H4TX19136 2012      4743.92
S2CA00286 2006       595.15
H0WI07051 2012      1965.96
H0NY02085 2010   1625923.74
H4TN06051 2008     86814.26
H8CA37137 2012    390071.78
...                     ...
H0MA06072 2012          NaN
H0AZ01267 2012          NaN
S4SC00083 2006      

In [196]:
cs1.head()

,,c1
cand_id,cycle,
H6FL19038,2006,281886.64
H2NY17071,2004,4500.00
H8TX27023,2008,12900.00
H8OH18088,2012,30.00
H0SC01071,2008,83183.76


In [199]:
#test
cs1.loc["H0SC01071", '2006']

/Users/adammorris/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app


c1
cand_id   cycle          
H0SC01071 2006   34484.85
          2006        NaN
          2006        NaN
          2006        NaN
          2006        NaN
          2006        NaN
          2006        NaN

In [200]:
agg_df = cs1.sum(level=['cand_id','cycle'])
print(agg_df)

                        c1
cand_id   cycle           
H0AK00089 2010      550.00
          2012         NaN
H0AK01038 2010     1569.23
H0AL00016 2010    19831.34
H0AL01030 2010     7054.34
          2012     3480.00
H0AL02087 2010         NaN
          2012    11727.82
          2014    12132.25
H0AL02103 2010     2197.40
H0AL02111 2010     4897.58
H0AL03184 2010       14.95
H0AL05049 2004         NaN
          2006         NaN
          2008         NaN
          2010     5818.75
          2012    11016.15
          2014     4592.00
H0AL05155 2010    56555.95
          2012         NaN
H0AL05163 2010    54616.04
          2012    54178.00
          2014    15420.65
H0AL05171 2010     8174.82
H0AL05189 2010     1082.20
H0AL05197 2010         NaN
          2012         NaN
H0AL07060 2004         NaN
          2006         NaN
          2008         NaN
...                    ...
S8TN00246 2008         NaN
S8TX00202 2008         NaN
          2010         NaN
S8TX00228 2008         NaN
S

In [203]:
#test
agg_df.loc['H0SC01071','2006']

c1    34484.85
Name: (H0SC01071, 2006), dtype: float64

In [204]:
#cs1 = cs1[pd.notnull(cs1['c1'])]

In [205]:
#cs1 = cs1.drop_duplicates()

In [206]:
#cs1.groupby(['cand_id','cycle'])['c1'].sum()

In [207]:
cs2 = catsort

In [208]:
cs2 = cs2.drop(['c1','c3','c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis =1)

In [209]:
cs2.set_index(['cand_id','cycle'], inplace=True)

In [210]:
print(cs2)

                 c2
cand_id   cycle    
H6FL19038 2006  NaN
H2NY17071 2004  NaN
H8TX27023 2008  NaN
H8OH18088 2012  NaN
H0SC01071 2008  NaN
S0CA00199 2010  NaN
S0MI00056 2014  NaN
H2GA12113 2012  NaN
H8NJ03156 2010  NaN
H6AZ08020 2006  NaN
H4WV03070 2014  NaN
H4WI04191 2006  NaN
H8VA10072 2008  NaN
H2MA01202 2012  NaN
H6WV03083 2006  NaN
H8WY00148 2012  NaN
H8WI01024 2014  NaN
H6TX09033 2004  NaN
H0IN07157 2012  NaN
H2MI11042 2014  NaN
H0FL12150 2010  NaN
H8AL05109 2012  NaN
H8VA11039 2010  NaN
H2MN08103 2012  NaN
H4TX19136 2012  NaN
S2CA00286 2006  NaN
H0WI07051 2012  NaN
H0NY02085 2010  NaN
H4TN06051 2008  NaN
H8CA37137 2012  NaN
...              ..
H0MA06072 2012  NaN
H0AZ01267 2012  NaN
S4SC00083 2006  NaN
H6TX09033 2004  NaN
S8NM00184 2014  NaN
H6OH15104 2014  NaN
H2TX21094 2012  NaN
S8ID00027 2010  NaN
H4PA08074 2006  NaN
S0NY00410 2012  NaN
S0NH00219 2008  NaN
H4TX09095 2008  NaN
H6HI02301 2006  NaN
S2MO00353 2006  NaN
H8OH01043 2008  NaN
H0NY27066 2010  NaN
H2AL05094 2014  NaN


In [211]:
#cs2 = catsort

In [212]:
cs2.head()

,,c2
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [213]:
#cs2 = cs2[pd.notnull(cs2['c2'])]

In [215]:
#test
cs2.loc["H2NY17071", '2004']

/Users/adammorris/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app


c2
cand_id   cycle          
H2NY17071 2004        NaN
          2004        NaN
          2004   12621.75
          2004        NaN
          2004        NaN
          2004        NaN
          2004        NaN
          2004        NaN
          2004        NaN
          2004        NaN
          2004        NaN

In [216]:
agg_df = cs2.sum(level=['cand_id','cycle'])
print(agg_df)

                       c2
cand_id   cycle          
H0AK00089 2010        NaN
          2012        NaN
H0AK01038 2010      88.93
H0AL00016 2010     909.80
H0AL01030 2010    5233.32
          2012      27.72
H0AL02087 2010        NaN
          2012        NaN
          2014        NaN
H0AL02103 2010        NaN
H0AL02111 2010        NaN
H0AL03184 2010    1441.91
H0AL05049 2004        NaN
          2006        NaN
          2008        NaN
          2010        NaN
          2012        NaN
          2014        NaN
H0AL05155 2010   15549.94
          2012        NaN
H0AL05163 2010    2343.79
          2012    8154.95
          2014    8540.32
H0AL05171 2010        NaN
H0AL05189 2010        NaN
H0AL05197 2010        NaN
          2012        NaN
H0AL07060 2004        NaN
          2006        NaN
          2008        NaN
...                   ...
S8TN00246 2008        NaN
S8TX00202 2008        NaN
          2010        NaN
S8TX00228 2008        NaN
S8VA00214 2004        NaN
          20

In [ ]:
#cs2.drop_duplicates()

In [217]:
#test
agg_df.loc['H0NC11118','2010']

c2    4715.72
Name: (H0NC11118, 2010), dtype: float64

In [218]:
cs2.head()

,,c2
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [219]:
#test
agg_df.loc['H2NY17071','2004']

c2    12621.75
Name: (H2NY17071, 2004), dtype: float64

In [ ]:
#cs2.groupby(['cand_id','cycle'])['c2'].sum()

In [220]:
cs3 = catsort

In [221]:
cs3 = cs3.drop(['c1', 'c2', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs3 = cs3[pd.notnull(cs3['c3'])]

In [222]:
cs3.head()

,cand_id,cycle,c3
0,H6FL19038,2006,NaN
1,H2NY17071,2004,NaN
2,H8TX27023,2008,NaN
3,H8OH18088,2012,NaN
4,H0SC01071,2008,NaN


In [223]:
cs3.set_index(['cand_id','cycle'], inplace=True)

In [ ]:
#cs3.groupby(['cand_id','cycle'])['c3'].sum()

In [224]:
cs3.head()

,,c3
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [ ]:
#cs3 = cs3.drop_duplicates()

In [225]:
agg_df = cs3.sum(level=['cand_id','cycle'])
print(agg_df)

                        c3
cand_id   cycle           
H0AK00089 2010         NaN
          2012         NaN
H0AK01038 2010         NaN
H0AL00016 2010       81.19
H0AL01030 2010      668.95
          2012      381.09
H0AL02087 2010         NaN
          2012    16964.09
          2014    16866.78
H0AL02103 2010         NaN
H0AL02111 2010      484.62
H0AL03184 2010         NaN
H0AL05049 2004         NaN
          2006         NaN
          2008         NaN
          2010         NaN
          2012         NaN
          2014         NaN
H0AL05155 2010   266006.98
          2012     4745.35
H0AL05163 2010    71452.12
          2012   121673.90
          2014    91996.02
H0AL05171 2010         NaN
H0AL05189 2010         NaN
H0AL05197 2010         NaN
          2012         NaN
H0AL07060 2004         NaN
          2006         NaN
          2008         NaN
...                    ...
S8TN00246 2008         NaN
S8TX00202 2008         NaN
          2010         NaN
S8TX00228 2008         NaN
S

In [226]:
#test
agg_df.loc['S8VA00214','2006']

c3    6585.79
Name: (S8VA00214, 2006), dtype: float64

In [227]:
cs4 = catsort

In [228]:
cs4 = cs4.drop(['c1', 'c2', 'c3', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs4 = cs4[pd.notnull(cs4['c4'])]

In [ ]:
#cs4.groupby(['cand_id','cycle'])['c4'].sum()

In [ ]:
#cs4 = cs4.drop_duplicates()

In [229]:
cs4.set_index(['cand_id','cycle'], inplace=True)

In [230]:
cs4.head()

,,c4
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [231]:
agg_df = cs4.sum(level=['cand_id','cycle'])
print(agg_df)

                        c4
cand_id   cycle           
H0AK00089 2010         NaN
          2012         NaN
H0AK01038 2010         NaN
H0AL00016 2010      465.00
H0AL01030 2010    27082.28
          2012     7257.71
H0AL02087 2010         NaN
          2012         NaN
          2014         NaN
H0AL02103 2010         NaN
H0AL02111 2010     8403.45
H0AL03184 2010     5000.00
H0AL05049 2004         NaN
          2006         NaN
          2008         NaN
          2010         NaN
          2012         NaN
          2014         NaN
H0AL05155 2010    20809.92
          2012         NaN
H0AL05163 2010   547321.04
          2012   165131.27
          2014    81481.66
H0AL05171 2010    40231.96
H0AL05189 2010         NaN
H0AL05197 2010      595.35
          2012         NaN
H0AL07060 2004         NaN
          2006         NaN
          2008         NaN
...                    ...
S8TN00246 2008         NaN
S8TX00202 2008         NaN
          2010         NaN
S8TX00228 2008         NaN
S

In [232]:
cs5 = catsort

In [233]:
cs5 = cs5.drop(['c1', 'c2', 'c3', 'c4', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs5 = cs5[pd.notnull(cs5['c5'])]

In [ ]:
#cs5.groupby(['cand_id','cycle'])['c5'].sum()

In [ ]:
#cs5 = cs5.drop_duplicates()

In [234]:
cs5.set_index(['cand_id','cycle'], inplace=True)

In [235]:
cs5.head()

,,c5
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [236]:
agg_df = cs5.sum(level=['cand_id','cycle'])
print(agg_df)

                      c5
cand_id   cycle         
H0AK00089 2010       NaN
          2012       NaN
H0AK01038 2010       NaN
H0AL00016 2010       NaN
H0AL01030 2010    1175.0
          2012       NaN
H0AL02087 2010       NaN
          2012       NaN
          2014       NaN
H0AL02103 2010       NaN
H0AL02111 2010       NaN
H0AL03184 2010       NaN
H0AL05049 2004       NaN
          2006       NaN
          2008       NaN
          2010       NaN
          2012       NaN
          2014       NaN
H0AL05155 2010       NaN
          2012       NaN
H0AL05163 2010   27000.0
          2012   17000.0
          2014   15000.0
H0AL05171 2010       NaN
H0AL05189 2010       NaN
H0AL05197 2010       NaN
          2012       NaN
H0AL07060 2004       NaN
          2006       NaN
          2008       NaN
...                  ...
S8TN00246 2008       NaN
S8TX00202 2008       NaN
          2010       NaN
S8TX00228 2008       NaN
S8VA00214 2004       NaN
          2006       NaN
          2008       NaN


In [237]:
#test
agg_df.loc['H0CA02138','2010']

c5    16750.0
Name: (H0CA02138, 2010), dtype: float64

In [238]:
cs5.head()

,,c5
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [239]:
cs6 = catsort

In [240]:
cs6 = cs6.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs6 = cs6[pd.notnull(cs6['c6'])]

In [ ]:
#cs6.groupby(['cand_id','cycle'])['c6'].sum()

In [ ]:
#cs6 = cs6.drop_duplicates()

In [241]:
cs6.set_index(['cand_id','cycle'], inplace=True)

In [242]:
cs6.head()

,,c6
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [243]:
agg_df = cs6.sum(level=['cand_id','cycle'])
print(agg_df)

                       c6
cand_id   cycle          
H0AK00089 2010     785.00
          2012        NaN
H0AK01038 2010        NaN
H0AL00016 2010    3177.55
H0AL01030 2010    1295.21
          2012        NaN
H0AL02087 2010        NaN
          2012        NaN
          2014        NaN
H0AL02103 2010        NaN
H0AL02111 2010    7489.74
H0AL03184 2010    2293.50
H0AL05049 2004        NaN
          2006        NaN
          2008        NaN
          2010        NaN
          2012        NaN
          2014        NaN
H0AL05155 2010        NaN
          2012        NaN
H0AL05163 2010   66857.04
          2012   16138.18
          2014   11833.66
H0AL05171 2010    5993.05
H0AL05189 2010        NaN
H0AL05197 2010        NaN
          2012        NaN
H0AL07060 2004        NaN
          2006        NaN
          2008        NaN
...                   ...
S8TN00246 2008        NaN
S8TX00202 2008        NaN
          2010        NaN
S8TX00228 2008        NaN
S8VA00214 2004        NaN
          20

In [244]:
cs6.head()

,,c6
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [245]:
#test
agg_df.loc['H0AR01109','2010']

c6    6671.16
Name: (H0AR01109, 2010), dtype: float64

In [246]:
cs7 = catsort

In [247]:
cs7 = cs7.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs7 = cs7[pd.notnull(cs7['c7'])]

In [ ]:
#cs7.groupby(['cand_id','cycle'])['c7'].sum()

In [ ]:
#cs7 = cs7.drop_duplicates()

In [248]:
cs7.set_index(['cand_id','cycle'], inplace=True)

In [249]:
cs7.head()

,,c7
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [250]:
agg_df = cs7.sum(level=['cand_id','cycle'])
print(agg_df)

                       c7
cand_id   cycle          
H0AK00089 2010        NaN
          2012        NaN
H0AK01038 2010        NaN
H0AL00016 2010    2192.99
H0AL01030 2010     583.75
          2012      56.58
H0AL02087 2010        NaN
          2012        NaN
          2014        NaN
H0AL02103 2010        NaN
H0AL02111 2010        NaN
H0AL03184 2010        NaN
H0AL05049 2004        NaN
          2006        NaN
          2008        NaN
          2010        NaN
          2012        NaN
          2014        NaN
H0AL05155 2010   47101.73
          2012        NaN
H0AL05163 2010    9362.66
          2012    9083.92
          2014    2350.00
H0AL05171 2010        NaN
H0AL05189 2010     205.00
H0AL05197 2010        NaN
          2012        NaN
H0AL07060 2004        NaN
          2006        NaN
          2008        NaN
...                   ...
S8TN00246 2008        NaN
S8TX00202 2008        NaN
          2010        NaN
S8TX00228 2008        NaN
S8VA00214 2004        NaN
          20

In [251]:
#test
agg_df.loc['H0AL02087','2010']

c7   NaN
Name: (H0AL02087, 2010), dtype: float64

In [252]:
cs8 = catsort

In [253]:
cs8 = cs8.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c9', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs8 = cs8[pd.notnull(cs8['c8'])]

In [ ]:
#cs8.groupby(['cand_id','cycle'])['c8'].sum()

In [ ]:
#cs8 = cs8.drop_duplicates()

In [254]:
cs8.head()

,cand_id,cycle,c8
0,H6FL19038,2006,NaN
1,H2NY17071,2004,NaN
2,H8TX27023,2008,NaN
3,H8OH18088,2012,NaN
4,H0SC01071,2008,NaN


In [255]:
cs8.set_index(['cand_id','cycle'], inplace=True)

In [256]:
cs8.head()

,,c8
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [257]:
agg_df = cs8.sum(level=['cand_id','cycle'])
print(agg_df)

                    c8
cand_id   cycle       
H0AK00089 2010     NaN
          2012     NaN
H0AK01038 2010     NaN
H0AL00016 2010     NaN
H0AL01030 2010     NaN
          2012     NaN
H0AL02087 2010     NaN
          2012     NaN
          2014     NaN
H0AL02103 2010     NaN
H0AL02111 2010     NaN
H0AL03184 2010     NaN
H0AL05049 2004     NaN
          2006     NaN
          2008     NaN
          2010     NaN
          2012     NaN
          2014     NaN
H0AL05155 2010     NaN
          2012     NaN
H0AL05163 2010   249.0
          2012     NaN
          2014     NaN
H0AL05171 2010     NaN
H0AL05189 2010     NaN
H0AL05197 2010     NaN
          2012     NaN
H0AL07060 2004     NaN
          2006     NaN
          2008     NaN
...                ...
S8TN00246 2008     NaN
S8TX00202 2008     NaN
          2010     NaN
S8TX00228 2008     NaN
S8VA00214 2004     NaN
          2006     NaN
          2008     NaN
          2012     NaN
          2014     NaN
S8VA00255 2008     NaN
          2

In [258]:
#test
agg_df.loc['S8WV00093','2008']

c8   NaN
Name: (S8WV00093, 2008), dtype: float64

In [259]:
cs9 = catsort

In [260]:
cs9 = cs9.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c10', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs9 = cs9[pd.notnull(cs9['c9'])]

In [ ]:
#cs9.groupby(['cand_id','cycle'])['c9'].sum()

In [ ]:
#cs9 = cs9.drop_duplicates()

In [261]:
cs9.set_index(['cand_id','cycle'], inplace=True)

In [262]:
agg_df = cs9.sum(level=['cand_id','cycle'])
print(agg_df)

                 c9
cand_id   cycle    
H0AK00089 2010  NaN
          2012  NaN
H0AK01038 2010  NaN
H0AL00016 2010  NaN
H0AL01030 2010  NaN
          2012  NaN
H0AL02087 2010  NaN
          2012  NaN
          2014  NaN
H0AL02103 2010  NaN
H0AL02111 2010  NaN
H0AL03184 2010  NaN
H0AL05049 2004  NaN
          2006  NaN
          2008  NaN
          2010  NaN
          2012  NaN
          2014  NaN
H0AL05155 2010  NaN
          2012  NaN
H0AL05163 2010  NaN
          2012  NaN
          2014  NaN
H0AL05171 2010  NaN
H0AL05189 2010  NaN
H0AL05197 2010  NaN
          2012  NaN
H0AL07060 2004  NaN
          2006  NaN
          2008  NaN
...              ..
S8TN00246 2008  NaN
S8TX00202 2008  NaN
          2010  NaN
S8TX00228 2008  NaN
S8VA00214 2004  NaN
          2006  NaN
          2008  NaN
          2012  NaN
          2014  NaN
S8VA00255 2008  NaN
          2010  NaN
S8VA00263 2008  NaN
          2012  NaN
          2014  NaN
S8WA00194 2006  NaN
          2008  NaN
          2010  NaN


In [263]:
cs9.head()

,,c9
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [264]:
#test
agg_df.loc['H0AL05163','2010']

c9   NaN
Name: (H0AL05163, 2010), dtype: float64

In [265]:
cs10 = catsort

In [266]:
cs10 = cs10.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c11', 'c12', 'c13','total'], axis = 1)

In [ ]:
#cs10 = cs10[pd.notnull(cs10['c10'])]

In [ ]:
#cs10.groupby(['cand_id','cycle'])['c10'].sum()

In [ ]:
#cs10 = cs10.drop_duplicates()

In [267]:
cs10.set_index(['cand_id','cycle'], inplace=True)

In [268]:
cs10.head()

,,c10
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [269]:
agg_df = cs10.sum(level=['cand_id','cycle'])
print(agg_df)

                   c10
cand_id   cycle       
H0AK00089 2010     NaN
          2012     NaN
H0AK01038 2010     NaN
H0AL00016 2010     NaN
H0AL01030 2010     NaN
          2012     NaN
H0AL02087 2010     NaN
          2012     NaN
          2014     NaN
H0AL02103 2010     NaN
H0AL02111 2010     NaN
H0AL03184 2010     NaN
H0AL05049 2004     NaN
          2006     NaN
          2008     NaN
          2010     NaN
          2012     NaN
          2014     NaN
H0AL05155 2010   610.0
          2012     NaN
H0AL05163 2010     NaN
          2012     NaN
          2014     NaN
H0AL05171 2010     NaN
H0AL05189 2010     NaN
H0AL05197 2010     NaN
          2012     NaN
H0AL07060 2004     NaN
          2006     NaN
          2008     NaN
...                ...
S8TN00246 2008     NaN
S8TX00202 2008     NaN
          2010     NaN
S8TX00228 2008     NaN
S8VA00214 2004     NaN
          2006     NaN
          2008     NaN
          2012     NaN
          2014     NaN
S8VA00255 2008     NaN
          2

In [270]:
#test
agg_df.loc['H0AL05155','2010']

c10    610.0
Name: (H0AL05155, 2010), dtype: float64

In [271]:
cs11 = catsort

In [272]:
cs11 = cs11.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c12', 'c13','total'], axis = 1)

In [273]:
cs11.head()

,cand_id,cycle,c11
0,H6FL19038,2006,NaN
1,H2NY17071,2004,NaN
2,H8TX27023,2008,NaN
3,H8OH18088,2012,NaN
4,H0SC01071,2008,NaN


In [ ]:
#cs11 = cs11[pd.notnull(cs11['c11'])]

In [ ]:
#cs11.groupby(['cand_id','cycle'])['c11'].sum()

In [ ]:
#cs11 = cs11.drop_duplicates()

In [274]:
cs11.set_index(['cand_id','cycle'], inplace=True)

In [275]:
cs11.head()

,,c11
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [276]:
agg_df = cs11.sum(level=['cand_id','cycle'])
print(agg_df)

                   c11
cand_id   cycle       
H0AK00089 2010     NaN
          2012     NaN
H0AK01038 2010     NaN
H0AL00016 2010     NaN
H0AL01030 2010     NaN
          2012     NaN
H0AL02087 2010     NaN
          2012     NaN
          2014     NaN
H0AL02103 2010     NaN
H0AL02111 2010     NaN
H0AL03184 2010     NaN
H0AL05049 2004     NaN
          2006     NaN
          2008     NaN
          2010     NaN
          2012     NaN
          2014     NaN
H0AL05155 2010     NaN
          2012     NaN
H0AL05163 2010     NaN
          2012     NaN
          2014     NaN
H0AL05171 2010     NaN
H0AL05189 2010     NaN
H0AL05197 2010   500.0
          2012     NaN
H0AL07060 2004     NaN
          2006     NaN
          2008     NaN
...                ...
S8TN00246 2008     NaN
S8TX00202 2008     NaN
          2010     NaN
S8TX00228 2008     NaN
S8VA00214 2004     NaN
          2006     NaN
          2008     NaN
          2012     NaN
          2014     NaN
S8VA00255 2008     NaN
          2

In [277]:
#test
agg_df.loc['H0AL05197','2010']

c11    500.0
Name: (H0AL05197, 2010), dtype: float64

In [278]:
cs12 = catsort

In [279]:
cs12 = cs12.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c13','total'], axis = 1)

In [280]:
cs12.head()

,cand_id,cycle,c12
0,H6FL19038,2006,NaN
1,H2NY17071,2004,NaN
2,H8TX27023,2008,NaN
3,H8OH18088,2012,NaN
4,H0SC01071,2008,NaN


In [ ]:
#cs12 = cs12[pd.notnull(cs12['c12'])]

In [ ]:
#cs12.groupby(['cand_id','cycle'])['c12'].sum()

In [ ]:
#cs12 = cs12.drop_duplicates()

In [281]:
cs12.set_index(['cand_id','cycle'], inplace=True)

In [282]:
cs12.head()

,,c12
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [283]:
agg_df = cs12.sum(level=['cand_id','cycle'])
print(agg_df)

                    c12
cand_id   cycle        
H0AK00089 2010      NaN
          2012      NaN
H0AK01038 2010      NaN
H0AL00016 2010    300.0
H0AL01030 2010      NaN
          2012      NaN
H0AL02087 2010      NaN
          2012      NaN
          2014      NaN
H0AL02103 2010      NaN
H0AL02111 2010      NaN
H0AL03184 2010      NaN
H0AL05049 2004      NaN
          2006      NaN
          2008      NaN
          2010      NaN
          2012    500.0
          2014   5000.0
H0AL05155 2010      NaN
          2012      NaN
H0AL05163 2010      NaN
          2012      NaN
          2014      NaN
H0AL05171 2010      NaN
H0AL05189 2010      NaN
H0AL05197 2010      NaN
          2012      NaN
H0AL07060 2004      NaN
          2006      NaN
          2008      NaN
...                 ...
S8TN00246 2008      NaN
S8TX00202 2008      NaN
          2010      NaN
S8TX00228 2008      NaN
S8VA00214 2004      NaN
          2006      NaN
          2008      NaN
          2012      NaN
          2014  

In [284]:
#test
agg_df.loc['H0AL05049', '2012']

c12    500.0
Name: (H0AL05049, 2012), dtype: float64

In [285]:
cs13 = catsort

In [286]:
cs13 = cs13.drop(['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12','total'], axis = 1)

In [287]:
cs13.head()

,cand_id,cycle,c13
0,H6FL19038,2006,NaN
1,H2NY17071,2004,NaN
2,H8TX27023,2008,NaN
3,H8OH18088,2012,NaN
4,H0SC01071,2008,NaN


In [ ]:
#cs13 = cs13[pd.notnull(cs13['c13'])]

In [ ]:
#cs13.groupby(['cand_id','cycle'])['c13'].sum()

In [ ]:
#cs13 = cs13.drop_duplicates()

In [288]:
cs13.set_index(['cand_id','cycle'], inplace=True)

In [289]:
cs13.head()

,,c13
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [290]:
agg_df = cs13.sum(level=['cand_id','cycle'])
print(agg_df)

                         c13
cand_id   cycle             
H0AK00089 2010     218666.06
          2012       3766.93
H0AK01038 2010     168119.58
H0AL00016 2010           NaN
H0AL01030 2010       1284.15
          2012           NaN
H0AL02087 2010    1403415.39
          2012     666262.07
          2014     541474.45
H0AL02103 2010     127314.17
H0AL02111 2010           NaN
H0AL03184 2010           NaN
H0AL05049 2004     467848.90
          2006     332285.59
          2008     454202.00
          2010      10023.55
          2012        735.00
          2014           NaN
H0AL05155 2010     518734.69
          2012       9324.78
H0AL05163 2010       1626.96
          2012       2321.95
          2014        215.00
H0AL05171 2010      18926.36
H0AL05189 2010     370262.34
H0AL05197 2010     756375.70
          2012     161419.77
H0AL07060 2004    1004122.38
          2006     405922.39
          2008     579570.41
...                      ...
S8TN00246 2008     155202.00
S8TX00202 2008

In [291]:
cs13.head()

,,c13
cand_id,cycle,
H6FL19038,2006,NaN
H2NY17071,2004,NaN
H8TX27023,2008,NaN
H8OH18088,2012,NaN
H0SC01071,2008,NaN


In [292]:
#test
agg_df.loc['S8WY00163','2008']

c13    15796.0
Name: (S8WY00163, 2008), dtype: float64

In [293]:
pr = exp

In [294]:
pr.head()

,state_abbreviation,district,incumbent,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,ge_winner_indicator,total_votes,cand_id,cmte_id,exp_sum,category,cycle
0,AL,7,(I),D,0.0,NaN,228518.0,NaN,W,228518,H0AL07060,C00347872,1363.96,NaN,2012
1,AL,7,(I),DEM,93586.0,NaN,133870.0,NaN,W,227456,H0AL07060,C00347872,1363.96,NaN,2012
2,AL,7,(I),D,58193.0,NaN,183408.0,NaN,W,241601,H0AL07060,C00347872,1363.96,NaN,2012
3,AL,7,(I),D,0.0,NaN,228518.0,NaN,W,228518,H0AL07060,C00347872,1204248.81,NaN,2010
4,AL,7,(I),DEM,93586.0,NaN,133870.0,NaN,W,227456,H0AL07060,C00347872,1204248.81,NaN,2010


In [295]:
pr = pr.drop(['cmte_id', 'exp_sum', 'category'], axis = 1)

In [ ]:
#exp= exp[pd.notnull(cs12['c12'])]

In [296]:
pr.head()

,state_abbreviation,district,incumbent,party,primary_votes,runoff_votes,general_votes,ge_runoff_election_votes,ge_winner_indicator,total_votes,cand_id,cycle
0,AL,7,(I),D,0.0,NaN,228518.0,NaN,W,228518,H0AL07060,2012
1,AL,7,(I),DEM,93586.0,NaN,133870.0,NaN,W,227456,H0AL07060,2012
2,AL,7,(I),D,58193.0,NaN,183408.0,NaN,W,241601,H0AL07060,2012
3,AL,7,(I),D,0.0,NaN,228518.0,NaN,W,228518,H0AL07060,2010
4,AL,7,(I),DEM,93586.0,NaN,133870.0,NaN,W,227456,H0AL07060,2010


In [ ]:
#cs10 = cs10.drop_duplicates()

In [297]:
pr.set_index(['cand_id','cycle'], inplace=True)

In [298]:
pr.head()

state_abbreviation district incumbent party  primary_votes  \
cand_id   cycle                                                              
H0AL07060 2012                  AL        7       (I)     D            0.0   
          2012                  AL        7       (I)   DEM        93586.0   
          2012                  AL        7       (I)     D        58193.0   
          2010                  AL        7       (I)     D            0.0   
          2010                  AL        7       (I)   DEM        93586.0   

                 runoff_votes  general_votes  ge_runoff_election_votes  \
cand_id   cycle                                                          
H0AL07060 2012            NaN       228518.0                       NaN   
          2012            NaN       133870.0                       NaN   
          2012            NaN       183408.0                       NaN   
          2010            NaN       228518.0                       NaN   
          2010            NaN       133870.0                       NaN   

                ge_winner_indicator  total_votes  
cand_id   cycle                                   
H0AL07060 2012                    W       228518  
          2012                    W       227456  
          2012                    W       241601  
          2010                    W       228518  
          2010                    W       227456

In [299]:
pr['primary_votes'].fillna(0, inplace = True)

In [300]:
pr['runoff_votes'].fillna(0, inplace = True)

In [301]:
pr['ge_runoff_election_votes'].fillna(0, inplace = True)

In [302]:
pr['general_votes'].fillna(0, inplace = True)

In [303]:
pr.head()

state_abbreviation district incumbent party  primary_votes  \
cand_id   cycle                                                              
H0AL07060 2012                  AL        7       (I)     D            0.0   
          2012                  AL        7       (I)   DEM        93586.0   
          2012                  AL        7       (I)     D        58193.0   
          2010                  AL        7       (I)     D            0.0   
          2010                  AL        7       (I)   DEM        93586.0   

                 runoff_votes  general_votes  ge_runoff_election_votes  \
cand_id   cycle                                                          
H0AL07060 2012            0.0       228518.0                       0.0   
          2012            0.0       133870.0                       0.0   
          2012            0.0       183408.0                       0.0   
          2010            0.0       228518.0                       0.0   
          2010            0.0       133870.0                       0.0   

                ge_winner_indicator  total_votes  
cand_id   cycle                                   
H0AL07060 2012                    W       228518  
          2012                    W       227456  
          2012                    W       241601  
          2010                    W       228518  
          2010                    W       227456

In [304]:
pr['primary_total'] = pr.primary_votes + pr.runoff_votes

In [305]:
pr.head()

state_abbreviation district incumbent party  primary_votes  \
cand_id   cycle                                                              
H0AL07060 2012                  AL        7       (I)     D            0.0   
          2012                  AL        7       (I)   DEM        93586.0   
          2012                  AL        7       (I)     D        58193.0   
          2010                  AL        7       (I)     D            0.0   
          2010                  AL        7       (I)   DEM        93586.0   

                 runoff_votes  general_votes  ge_runoff_election_votes  \
cand_id   cycle                                                          
H0AL07060 2012            0.0       228518.0                       0.0   
          2012            0.0       133870.0                       0.0   
          2012            0.0       183408.0                       0.0   
          2010            0.0       228518.0                       0.0   
          2010            0.0       133870.0                       0.0   

                ge_winner_indicator  total_votes  primary_total  
cand_id   cycle                                                  
H0AL07060 2012                    W       228518            0.0  
          2012                    W       227456        93586.0  
          2012                    W       241601        58193.0  
          2010                    W       228518            0.0  
          2010                    W       227456        93586.0

In [306]:
pr['general_total'] = pr.general_votes + pr.ge_runoff_election_votes

In [307]:
pr.head()

state_abbreviation district incumbent party  primary_votes  \
cand_id   cycle                                                              
H0AL07060 2012                  AL        7       (I)     D            0.0   
          2012                  AL        7       (I)   DEM        93586.0   
          2012                  AL        7       (I)     D        58193.0   
          2010                  AL        7       (I)     D            0.0   
          2010                  AL        7       (I)   DEM        93586.0   

                 runoff_votes  general_votes  ge_runoff_election_votes  \
cand_id   cycle                                                          
H0AL07060 2012            0.0       228518.0                       0.0   
          2012            0.0       133870.0                       0.0   
          2012            0.0       183408.0                       0.0   
          2010            0.0       228518.0                       0.0   
          2010            0.0       133870.0                       0.0   

                ge_winner_indicator  total_votes  primary_total  general_total  
cand_id   cycle                                                                 
H0AL07060 2012                    W       228518            0.0       228518.0  
          2012                    W       227456        93586.0       133870.0  
          2012                    W       241601        58193.0       183408.0  
          2010                    W       228518            0.0       228518.0  
          2010                    W       227456        93586.0       133870.0

In [ ]:
#pd.merge(pr,cs1, how = 'left', on = 'cand_id')

In [308]:
RS1 = pr.join(cs1)

In [310]:
RS2 = RS1.join(cs2)

In [311]:
RS3 = RS2.join(cs3)

In [ ]:
RS4 = RS3.join(cs4)

In [ ]:
#cm1 = pd.merge(cs1, cs2, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs3, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs4, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs5, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs6, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs7, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs8, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs9, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs10, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs11, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = pd.merge(csm1, cs12, on=['cand_id','cycle'], how='left')

In [ ]:
#csm1 = csm1.drop(['c13'], axis = 1)

In [ ]:
#csm1 = csm1.drop_duplicates()

In [ ]:
csm1.head(200)

In [ ]:
#cs12.groupby(['cand_id','cycle'])['c12'].sum()

In [ ]:
df.head()

In [ ]:
df = election_spend

In [ ]:
df.drop('id', axis=1, inplace=True)

In [ ]:
df.drop('cand_name', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.count()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.head(4)

In [ ]:
df.count()

In [ ]:
df.loc[:'H6FL19038']

In [ ]:
df = df[df.total_votes != 0]

In [ ]:
df.head()

In [ ]:
Expense = df[["cand_id","cand_ici","cycle","c1","c2","c3","c4","c5","c6","c7","c8","c9","c10","c11","c12","c13"]]

In [ ]:
Other = df[["cand_id","cand_pty_affiliation","cand_office_st","cand_office","cand_office_district","cand_ici","cycle","total_votes"]]

In [ ]:
Expense.head(10)

In [ ]:
Expense_Update = Expense.drop_duplicates()

In [ ]:
Expense_Update.head()

In [ ]:
results.head()

In [ ]:
df1 = results

In [ ]:
df1.fec_id.apply(str)

In [ ]:
df2 = df1[pd.notnull(df1['fec_id'])]

In [ ]:
df2.head(20)

In [ ]:
df2['incumbent']=df2['incumbent'].fillna(0)

In [ ]:
df2['primary_votes']=df2['primary_votes'].fillna(0)

In [ ]:
df2['runoff_votes']=df2['runoff_votes'].fillna(0)

In [ ]:
df2['general_votes']=df2['general_votes'].fillna(0)

In [ ]:
df2['ge_runoff_election_votes']=df2['ge_runoff_election_votes'].fillna(0)

In [ ]:
df2.head()

In [ ]:
df2['ge_winner_indicator']=df2['ge_winner_indicator'].fillna(0)

In [ ]:
df2.head(20)

In [ ]:
df3 = df2.replace("W",1)

In [ ]:
df3.head(20)

In [ ]:
df4 = df3.replace("(I)",1)

In [ ]:
df4.head(20)

In [ ]:
results_clean = df4.drop_duplicates()

In [ ]:
results_clean.count()

In [ ]:
results_clean.drop('ge_runoff_election_votes', axis=1, inplace=True)

In [ ]:
results_clean.drop('ge_winner_indicator', axis=1, inplace=True)

In [ ]:
results_clean.drop('blank', axis=1, inplace=True)

In [ ]:
results_clean.head(20)

In [ ]:
from pandas.tools.plotting import parallel_coordinates
parallel_coordinates(data, class_column='label')

In [ ]:
from sklearn.model_selection import train_test_split as tts 

X = data[[
    '??', '??', '??', '??', '??', '??', '??'
]]
y = data['label']

print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)